In [ ]:
#freqdicts pour dépendants ou head des lemmes pivot, sans et avec filtre

In [43]:
output_ep1_s = "/home/port-pret-etu01/Documents/LATTICE/dependencies/dep (what is dep of pain)/dep_ep1_s.txt"
output_ep2_s = "/home/port-pret-etu01/Documents/LATTICE/dependencies/dep (what is dep of pain)/dep_ep2_s.txt"
output_ep3_s = "/home/port-pret-etu01/Documents/LATTICE/dependencies/dep (what is dep of pain)/dep_ep3_s.txt"

output_ep1_a = "/home/port-pret-etu01/Documents/LATTICE/dependencies/dep (what is dep of pain)/dep_ep1_a.txt"
output_ep2_a = "/home/port-pret-etu01/Documents/LATTICE/dependencies/dep (what is dep of pain)/dep_ep2_a.txt"
output_ep3_a = "/home/port-pret-etu01/Documents/LATTICE/dependencies/dep (what is dep of pain)/dep_ep3_a.txt"

output_ep1_y = "/home/port-pret-etu01/Documents/LATTICE/dependencies/dep (what is dep of pain)/dep_ep1_y.txt"
output_ep2_y = "/home/port-pret-etu01/Documents/LATTICE/dependencies/dep (what is dep of pain)/dep_ep2_y.txt"
output_ep3_y = "/home/port-pret-etu01/Documents/LATTICE/dependencies/dep (what is dep of pain)/dep_ep3_y.txt"

o_ep1_s = "/home/port-pret-etu01/Documents/LATTICE/dependencies/DEP (pain dep  of)/DEP_ep1_s.txt"
o_ep2_s = "/home/port-pret-etu01/Documents/LATTICE/dependencies/DEP (pain dep  of)/DEP_ep2_s.txt"
o_ep3_s = "/home/port-pret-etu01/Documents/LATTICE/dependencies/DEP (pain dep  of)/DEP_ep3_s.txt"

o_ep1_a = "/home/port-pret-etu01/Documents/LATTICE/dependencies/DEP (pain dep  of)/DEP_ep1_a.txt"
o_ep2_a = "/home/port-pret-etu01/Documents/LATTICE/dependencies/DEP (pain dep  of)/DEP_ep2_a.txt"
o_ep3_a = "/home/port-pret-etu01/Documents/LATTICE/dependencies/DEP (pain dep  of)/DEP_ep3_a.txt"

o_ep1_y = "/home/port-pret-etu01/Documents/LATTICE/dependencies/DEP (pain dep  of)/DEP_ep1_y.txt"
o_ep2_y = "/home/port-pret-etu01/Documents/LATTICE/dependencies/DEP (pain dep  of)/DEP_ep2_y.txt"
o_ep3_y = "/home/port-pret-etu01/Documents/LATTICE/dependencies/DEP (pain dep  of)/DEP_ep3_y.txt"


outputs_s = [output_ep1_s, output_ep2_s, output_ep3_s] 
outputs_a = [output_ep1_a, output_ep2_a, output_ep3_a]
outputs_y = [output_ep1_y, output_ep2_y, output_ep3_y]

o_s = [o_ep1_s, o_ep2_s, o_ep3_s] 
o_a = [o_ep1_a, o_ep2_a, o_ep3_a]
o_y = [o_ep1_y, o_ep2_y, o_ep3_y]

In [44]:
#code pour tableaux sans filtre, top 10 token

In [45]:
import pandas as pd
from collections import Counter
from tabulate import tabulate
import os
import json

with open("/home/port-pret-etu01/Documents/LATTICE/méta/output/total_pivot_counts.json", "r") as f:
    pivot_totals = json.load(f)

def detect_epoque(path):
    if "ep1" in path:
        return "ep1"
    elif "ep2" in path:
        return "ep2"
    elif "ep3" in path:
        return "ep3"
    else:
        raise ValueError(f"Époque inconnue pour le chemin : {path}")

categories = {
    "s": {"output": outputs_s, "o": o_s},
    "a": {"output": outputs_a, "o": o_a},
    "y": {"output": outputs_y, "o": o_y}
}

groupes = {
    "s": ["pain", "pains"],
    "a": ["painful"],
    "y": ["painfully"]
}

def obtenir_total_lemma(epoque, lemmes):
    return sum(pivot_totals.get(epoque, {}).get(l, 0) for l in lemmes)

def analyser_categorie(nom_categorie, fichiers_dict):
    lemmes_courants = groupes[nom_categorie]

    for type_fichier, fichiers in fichiers_dict.items():
        if not fichiers:
            continue

        dossier_sortie = os.path.dirname(fichiers[0])
        chemin_sortie = os.path.join(dossier_sortie, f"auswertung_{nom_categorie}_{type_fichier}.txt")

        with open(chemin_sortie, "w", encoding="utf-8") as out:
            for chemin_fichier in fichiers:
                with open(chemin_fichier, "r", encoding="utf-8") as f:
                    lignes = [ligne.strip() for ligne in f if ligne.strip()]

                if not lignes:
                    out.write(f"Fichier : {chemin_fichier}\nAucun lemme trouvé.\n\n")
                    continue

                ep = detect_epoque(chemin_fichier)
                total_lemmas = obtenir_total_lemma(ep, lemmes_courants)
                freq = Counter(lignes)
                top_50 = freq.most_common(50)

                out.write(f"Fichier : {chemin_fichier}\n")
                out.write(f"Total pour {' + '.join(lemmes_courants)} dans {ep} : {total_lemmas}\n\n")

                df = pd.DataFrame(top_50, columns=["Lemme", "Fréquence"])
                df["Part / total pivot (%)"] = df["Fréquence"].apply(
                    lambda x: round((x / total_lemmas) * 100, 2) if total_lemmas > 0 else "N/A"
                )

                tableau = tabulate(df, headers="keys", tablefmt="fancy_grid", showindex=False)
                out.write(tableau + "\n\n" + "=" * 80 + "\n\n")

for cat, fichiers_dict in categories.items():
    analyser_categorie(cat, fichiers_dict)


In [46]:
#code pour tableaux avec filtre (be, were, is, are...), top 10 token

In [47]:
import pandas as pd
from collections import Counter
from tabulate import tabulate
import os
import json

with open("/home/port-pret-etu01/Documents/LATTICE/méta/output/total_pivot_counts.json", "r") as f:
    pivot_totals = json.load(f)

stop_words = {
    "is", "are", "be", "were", "'s", "been", "was", "-", "the", "the of",
    "of", "some", "a", "the in", "a in", "such", "in", "any", ".", "no"
}

pronoun_keywords = {
    "his", "her", "our", "ours", "hers", "him", "she", "he", "we", "i", "my",
    "thy", "their", "you", "your", "mine", "-"
}

outputs_dict = {
    "s": outputs_s,
    "a": outputs_a,
    "y": outputs_y
}

o_dict = {
    "s": o_s,
    "a": o_a,
    "y": o_y
}

groupes = {
    "s": ["pain", "pains"],
    "a": ["painful"],
    "y": ["painfully"]
}

TOP_N_FREQ_LIST = 400

def detect_epoque(path):
    if "ep1" in path:
        return "ep1"
    elif "ep2" in path:
        return "ep2"
    elif "ep3" in path:
        return "ep3"
    else:
        raise ValueError(f"Époque inconnue pour le chemin : {path}")

def obtenir_total_lemma(epoque, lemmes):
    return sum(pivot_totals.get(epoque, {}).get(l, 0) for l in lemmes)

def analyse_fichiers_filtrés(liste_fichiers, nom_categorie):
    lemmes_courants = groupes[nom_categorie[-1]]
    dossiers = {}

    for fichier in liste_fichiers:
        dossier = os.path.dirname(fichier)
        if dossier not in dossiers:
            dossiers[dossier] = []
        dossiers[dossier].append(fichier)

    for dossier, fichiers in dossiers.items():
        chemin_sortie = os.path.join(dossier, f"auswertung_filter_{nom_categorie}.txt")
        with open(chemin_sortie, "w", encoding="utf-8") as out:
            for fichier in fichiers:
                ep = detect_epoque(fichier)
                total_pivot = obtenir_total_lemma(ep, lemmes_courants)

                with open(fichier, "r", encoding="utf-8") as f:
                    tokens = [ligne.strip() for ligne in f if ligne.strip()]

                tokens_filtrés = []
                for token in tokens:
                    token_lc = token.lower().strip()
                    if token_lc in stop_words:
                        continue
                    trouvé = None
                    for pronom in pronoun_keywords:
                        if token_lc == pronom or token_lc.endswith(" " + pronom) or token_lc.startswith(pronom + " ") or f" {pronom} " in token_lc:
                            trouvé = pronom
                            break
                    if trouvé:
                        tokens_filtrés.append(trouvé)
                    else:
                        tokens_filtrés.append(token_lc)

                tokens_stop = [t for t in tokens if t.lower() in stop_words]

                total_tokens = len(tokens)
                total_stop = len(tokens_stop)
                total_après_filtrage = total_pivot - total_stop if total_pivot - total_stop > 0 else 1

                out.write(f"Fichier : {fichier}\n")
                out.write(f"Total pour {' + '.join(lemmes_courants)} dans {ep} : {total_pivot}\n")
                out.write(f"Nombre de stopwords : {total_stop} ({round((total_stop / total_pivot) * 100, 2) if total_pivot else 0}%)\n")
                out.write(f"Total après filtrage : {total_après_filtrage}\n\n")

                if not tokens_filtrés:
                    out.write("Aucun token après filtrage.\n\n")
                    continue

                freq = Counter(tokens_filtrés)
                top_10 = freq.most_common(10)

                df = pd.DataFrame(top_10, columns=["Token", "Fréquence"])
                df["Part / FILTRÉ (%)"] = df["Fréquence"].apply(lambda x: round((x / total_après_filtrage) * 100, 2))
                df["Part / TOTAL Pivot (%)"] = df["Fréquence"].apply(lambda x: round((x / total_pivot) * 100, 2) if total_pivot > 0 else 0)

                tableau = tabulate(df, headers="keys", tablefmt="fancy_grid", showindex=False)
                out.write(tableau + "\n\n")

                out.write("Liste complète des tokens les plus fréquents :\n")
                top_freqs = freq.most_common(TOP_N_FREQ_LIST)
                for token, count in top_freqs:
                    out.write(f"{token}: {count}\n")

                out.write("\n" + "=" * 80 + "\n\n")

for cat in ["s", "a", "y"]:
    analyse_fichiers_filtrés(outputs_dict[cat], f"output_{cat}")
    analyse_fichiers_filtrés(o_dict[cat], f"o_{cat}")
